In [23]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle
import math

In [2]:
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time

In [541]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [163]:
# Get season stats for 16-17 season to get starting elo for training data
nba_16_17_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2016-17&SeasonType=Regular%20Season"

In [164]:
# visit website
browser.visit(nba_16_17_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [165]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [166]:
df_init = df.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
game_dates = df_init['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]

In [167]:
date_index=0
newElo = {}
eloScores = []
for row in df_init.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_list[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_list[team1], elo_list[team2], 20, outcome)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_list[team2], elo_list[team1], 20, outcome)
    newElo[row.Team] = eloUpdate
# Update for last date
for key in newElo:
    elo_list[key] = newElo[key]

In [169]:
db.nba_team_elo_training.insert_one(elo_list)

In [4]:
# Url to get all boxscores for 2017-2018 regular season games
nba_17_18_trad_boxscores_url = "https://stats.nba.com/teams/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2017-18&SeasonType=Regular%20Season"

In [8]:
# create splinter browser
def init_browser(noShowBrowser):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=noShowBrowser)
    return browser

In [542]:
browser = init_browser(False)

In [225]:
# visit website
browser.visit(nba_17_18_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df_adv = df[0].drop(columns=['Season'])
# Rename column names
df_adv.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df_adv['GameDate'] = df_adv['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [226]:
df_adv.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE'],
      dtype='object')

In [229]:
# visit website
browser.visit(nba_17_18_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_ff = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%', 'OppeFG%', 'OppFTARate', 'OppTOV%', 'OppOREB%'], axis=1)

In [230]:
df_ff.columns

Index(['Team', 'GameDate', 'FTARate'], dtype='object')

In [232]:
# visit website
browser.visit(nba_17_18_trad_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_trad = df.drop(['MIN', 'MatchUp', 'W/L', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'PF', '+/-'], axis=1)

In [233]:
df_trad.columns

Index(['Team', 'GameDate', 'PTS'], dtype='object')

In [234]:
result = pd.merge(df_adv, df_ff, on=['Team', 'GameDate'])
result.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate'],
      dtype='object')

In [235]:
result_final = pd.merge(result, df_trad, on=['Team', 'GameDate'])
result_final.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'PTS'],
      dtype='object')

In [28]:
teams = result_final['Team'].unique()

In [29]:
myteam = [team for team in teams]

In [30]:
myteam.sort()

In [236]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [37]:
db.nba_team_elo_training.insert_one(elo_list)

In [261]:
result_sorted = result_final.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
#result_sorted['HomeGame'] = result_sorted['MatchUp'].apply(lambda x: 1 if x.split(' ')[1] == 'vs.' else 0)

In [64]:
def eloProbability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))
def EloRating(homeTeam, roadTeam, K, homeTeamWins):
    # To calculate the Winning 
    # Probability of Player B 
    Pb = eloProbability(homeTeam, roadTeam) 

    # To calculate the Winning 
    # Probability of Player A 
    Pa = eloProbability(roadTeam, homeTeam) 

    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if (homeTeamWins == True) : 
        homeTeam = homeTeam + K * (1 - Pa) 
        roadTeam = roadTeam + K * (0 - Pb) 


    # Case -2 When Player B wins 
    # Updating the Elo Ratings 
    else : 
        homeTeam = homeTeam + K * (0 - Pa)
        roadTeam = roadTeam + K * (1 - Pb)
    return homeTeam, roadTeam

In [66]:
team1 = 1000
team2 = 1000

In [67]:
team1, team2 = EloRating(team1, team2, 20, False)

In [69]:
team2

1010.0

In [262]:
game_dates = result_sorted['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]
game_dates[0]

numpy.datetime64('2017-10-17T00:00:00.000000000')

In [81]:
game_dates[1] == np.datetime64(result_sorted.iloc[0]['GameDate'])

False

In [76]:
result_sorted.iloc[0]['GameDate']

Timestamp('2017-10-17 00:00:00')

In [264]:
temp_df = result_sorted.copy()

In [265]:
temp_df.dtypes

Team                object
MatchUp             object
GameDate    datetime64[ns]
W/L                 object
MIN                  int64
OffRtg             float64
DefRtg             float64
NetRtg             float64
AST%               float64
AST/TO             float64
ASTRatio           float64
OREB%              float64
DREB%              float64
REB%               float64
TOV%               float64
eFG%               float64
TS%                float64
PACE               float64
PIE                float64
FTARate            float64
PTS                  int64
dtype: object

In [266]:
# Init ELO for training season
elo_list = {}
for team in myteam:
    elo_list[team] = 1500.0

In [267]:
date_index=0
newElo = {}
eloScores = []
homeTeam = []
roadTeam = []
for row in temp_df.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_list[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_list[team1], elo_list[team2], 20, outcome)
        homeTeam.append(team1)
        roadTeam.append(team2)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_list[team2], elo_list[team1], 20, outcome)
        roadTeam.append(team1)
        homeTeam.append(team2)
        
    newElo[row.Team] = eloUpdate
    eloScores.append(eloUpdate)
# Update for last date
for key in newElo:
    elo_list[key] = newElo[key]

In [544]:
elo_list

{'ATL': 1379.585954026032,
 'BKN': 1404.8281020435775,
 'BOS': 1581.6442164590449,
 'CHA': 1455.7326971389414,
 'CHI': 1369.5264989320076,
 'CLE': 1569.2570084824733,
 'DAL': 1349.9955539253224,
 'DEN': 1567.3908510170115,
 'DET': 1469.2436740169394,
 'GSW': 1585.218097082357,
 'HOU': 1712.3692616813594,
 'IND': 1579.9535417247766,
 'LAC': 1514.4717110441677,
 'LAL': 1463.3333766030464,
 'MEM': 1315.91711181331,
 'MIA': 1515.8396295778298,
 'MIL': 1518.023873793636,
 'MIN': 1538.5720049287456,
 'NOP': 1588.4605069805805,
 'NYK': 1369.6778447449904,
 'OKC': 1579.0910067247698,
 'ORL': 1352.7914611902586,
 'PHI': 1638.0820435490868,
 'PHX': 1305.2825248358438,
 'POR': 1592.5772517125845,
 'SAC': 1399.0131403590187,
 'SAS': 1541.9868737873699,
 'TOR': 1642.7968580395868,
 'UTA': 1613.6622117595898,
 'WAS': 1485.675112025742}

In [245]:
len(eloScores)

2460

In [246]:
len(temp_df)

2460

In [269]:
temp_df['ELO'] = eloScores
temp_df['HomeTeam'] = homeTeam
temp_df['RoadTeam'] = roadTeam

In [270]:
temp_df.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'PTS', 'ELO',
       'HomeTeam', 'RoadTeam'],
      dtype='object')

In [271]:
home_df = temp_df[temp_df['Team'] == temp_df['HomeTeam']]
road_df = temp_df[temp_df['Team'] == temp_df['RoadTeam']]

In [278]:
game_df = pd.merge(home_df, road_df, how='inner', on=['HomeTeam', 'RoadTeam', 'GameDate'], suffixes=('_h', '_r'), validate="one_to_one")

In [279]:
game_df.columns

Index(['Team_h', 'MatchUp_h', 'GameDate', 'W/L_h', 'MIN_h', 'OffRtg_h',
       'DefRtg_h', 'NetRtg_h', 'AST%_h', 'AST/TO_h', 'ASTRatio_h', 'OREB%_h',
       'DREB%_h', 'REB%_h', 'TOV%_h', 'eFG%_h', 'TS%_h', 'PACE_h', 'PIE_h',
       'FTARate_h', 'PTS_h', 'ELO_h', 'HomeTeam', 'RoadTeam', 'Team_r',
       'MatchUp_r', 'W/L_r', 'MIN_r', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r',
       'AST%_r', 'AST/TO_r', 'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r',
       'TOV%_r', 'eFG%_r', 'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r',
       'ELO_r'],
      dtype='object')

In [280]:
game_df = game_df.drop(['Team_h', 'Team_r', 'MatchUp_h', 'MatchUp_r', 'W/L_h', 'W/L_r', 'MIN_h', 'MIN_r'], axis=1)

In [282]:
game_df.columns

Index(['GameDate', 'OffRtg_h', 'DefRtg_h', 'NetRtg_h', 'AST%_h', 'AST/TO_h',
       'ASTRatio_h', 'OREB%_h', 'DREB%_h', 'REB%_h', 'TOV%_h', 'eFG%_h',
       'TS%_h', 'PACE_h', 'PIE_h', 'FTARate_h', 'PTS_h', 'ELO_h', 'HomeTeam',
       'RoadTeam', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r', 'AST%_r', 'AST/TO_r',
       'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r', 'TOV%_r', 'eFG%_r',
       'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r', 'ELO_r'],
      dtype='object')

In [283]:
db.score_pred_training_data.drop()

In [284]:
# export to json
items = game_df.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [285]:
db.score_pred_training_data.insert_many(items_db)

In [185]:
db.nba_team_elo_testing.insert_one(elo_list)

In [545]:
temp = db.nba_team_elo_testing.find()

In [546]:
temp = list(temp)

In [547]:
for i in temp:
    i.pop('_id', None)

In [548]:
elo_testing = temp[0]

In [549]:
elo_testing

{'ATL': 1375.7075741834688,
 'BKN': 1385.7473632514852,
 'BOS': 1593.1622149515758,
 'CHA': 1447.7280767309485,
 'CHI': 1367.1005274309066,
 'CLE': 1570.4270859757785,
 'DAL': 1343.0240681009527,
 'DEN': 1571.0319534263213,
 'DET': 1461.7873401532459,
 'GSW': 1619.4570676450153,
 'HOU': 1727.4508941343163,
 'IND': 1581.231834171123,
 'LAC': 1524.2948631384638,
 'LAL': 1451.6639776614604,
 'MEM': 1312.794183162584,
 'MIA': 1518.578999882534,
 'MIL': 1518.6617359287,
 'MIN': 1533.1207602745035,
 'NOP': 1588.8232069597443,
 'NYK': 1356.0233382357949,
 'OKC': 1586.408713208459,
 'ORL': 1336.8270182401138,
 'PHI': 1624.0357372854346,
 'PHX': 1286.05099854899,
 'POR': 1598.9654622637888,
 'SAC': 1390.6202815200002,
 'SAS': 1558.4988631578274,
 'TOR': 1652.5998449378783,
 'UTA': 1626.4434841314753,
 'WAS': 1491.7325313071099}

In [196]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [551]:
temp = db.score_pred_training_data.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df_train = pd.DataFrame(temp)

In [286]:
game_df.columns

Index(['GameDate', 'OffRtg_h', 'DefRtg_h', 'NetRtg_h', 'AST%_h', 'AST/TO_h',
       'ASTRatio_h', 'OREB%_h', 'DREB%_h', 'REB%_h', 'TOV%_h', 'eFG%_h',
       'TS%_h', 'PACE_h', 'PIE_h', 'FTARate_h', 'PTS_h', 'ELO_h', 'HomeTeam',
       'RoadTeam', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r', 'AST%_r', 'AST/TO_r',
       'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r', 'TOV%_r', 'eFG%_r',
       'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r', 'ELO_r'],
      dtype='object')

In [552]:
df_train.head()

,AST%_h,AST%_r,AST/TO_h,AST/TO_r,ASTRatio_h,ASTRatio_r,DREB%_h,DREB%_r,DefRtg_h,DefRtg_r,...,PTS_r,REB%_h,REB%_r,RoadTeam,TOV%_h,TOV%_r,TS%_h,TS%_r,eFG%_h,eFG%_r
0,79.1,59.6,2.00,2.15,23.9,18.9,66.7,84.6,119.6,118.6,...,122,45.2,54.8,HOU,16.7,12.7,67.8,57.9,63.8,56.2
1,50.0,66.7,1.12,2.00,14.8,17.9,76.8,77.1,100.0,104.1,...,99,51.9,48.1,BOS,17.3,12.1,54.3,50.0,48.8,45.5
2,54.7,48.9,2.07,1.10,17.6,14.4,68.6,63.0,113.9,119.7,...,131,52.4,47.6,BKN,12.0,17.4,60.3,60.6,56.4,54.3
3,35.7,50.0,0.63,1.22,7.9,15.3,53.2,74.2,122.8,74.5,...,124,37.6,62.4,POR,15.7,17.8,39.2,60.6,35.4,56.7
4,58.5,55.2,2.67,0.94,18.0,13.2,84.8,80.4,90.9,103.0,...,90,49.0,51.0,CHA,9.1,17.2,50.4,52.5,47.4,45.9


In [770]:
#X_train = game_df.loc[:,['OffRtg_h', 'DefRtg_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h',
#                         'OffRtg_r', 'DefRtg_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']]
X_train = df_train.loc[:,['DefRtg_h', 'PIE_h', 'FTARate_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h',
                          'DefRtg_r', 'PIE_r', 'FTARate_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']]
y_h = df_train['PTS_h']
y_r = df_train['PTS_r']
y_train = df_train[['PTS_h', 'PTS_r']]

In [698]:
X_train.shape

(1230, 16)

In [699]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)

In [701]:
homeLinearModel = LinearRegression()
roadLinearModel = LinearRegression()

In [671]:
#linearModel = LinearRegression()

In [672]:
#linearModel.fit(X_train_scaled, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [673]:
#linearModel.score(X_train_scaled, y)

0.9421288632690912

In [702]:
homeLinearModel.fit(X_train_scaled, y_h)
roadLinearModel.fit(X_train_scaled, y_r)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [703]:
homeLinearModel.score(X_train_scaled, y_h)

0.9397954244572464

In [704]:
roadLinearModel.score(X_train_scaled, y_r)

0.9442859315186201

In [705]:
homeLinearModel.coef_

array([  1.13179316,   3.7218134 ,  -0.0675043 , -17.346949  ,
        -0.72022729,   1.08607602,   2.51939399,   0.12453364,
         9.27389857,   4.02541127,   0.09482707, -18.04671709,
         0.60067408,  -1.04075873,   2.51939399,  -0.0449362 ])

In [706]:
roadLinearModel.coef_

array([ 1.01926916e+01, -6.00555603e+01,  2.86892045e-02, -3.85333663e+01,
        2.90727045e-01, -5.30290354e-01,  2.50098758e+00,  1.82418571e-02,
        7.96343309e-01, -5.98717577e+01, -1.04515988e-02, -3.79356090e+01,
       -3.68491660e-01,  4.62271315e-01,  2.50098758e+00,  2.15941605e-02])

In [655]:
from sklearn.linear_model import BayesianRidge

In [771]:
homeBay = BayesianRidge(verbose=1)
roadBay = BayesianRidge(verbose=1)

In [772]:
X_train_scaled.shape

(1230, 16)

In [773]:
homeBay.fit(X_train_scaled, y_h)
roadBay.fit(X_train_scaled, y_r)

Convergence after  4  iterations
Convergence after  4  iterations


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=1)

In [774]:
homeBay.score(X_train_scaled, y_h)

0.9397818510165548

In [775]:
roadBay.score(X_train_scaled, y_r)

0.9442227910412742

In [666]:
from sklearn.linear_model import HuberRegressor

In [716]:
huberModelHome = HuberRegressor()
huberModelRoad = HuberRegressor()

In [717]:
huberModelHome.fit(X_train_scaled, y_h)
huberModelRoad.fit(X_train_scaled, y_r)

HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
        tol=1e-05, warm_start=False)

In [718]:
huberModelHome.score(X_train_scaled, y_h)

0.9372990759987583

In [719]:
huberModelRoad.score(X_train_scaled, y_r)

0.9419763447686365

In [721]:
from sklearn import linear_model

In [723]:
ardModelHome = linear_model.ARDRegression(verbose=1)
ardModelRoad = linear_model.ARDRegression(verbose=1)

In [724]:
ardModelHome.fit(X_train_scaled, y_h)
ardModelRoad.fit(X_train_scaled, y_r)

Converged after 134 iterations
Converged after 88 iterations


ARDRegression(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, threshold_lambda=10000.0, tol=0.001, verbose=1)

In [725]:
ardModelHome.score(X_train_scaled, y_h)

0.9393773505357557

In [727]:
ardModelRoad.score(X_train_scaled, y_r)

0.9437573431084013

In [741]:
sgdModelHome = linear_model.SGDRegressor(loss='huber', penalty='l2', learning_rate='optimal', max_iter=1000, tol=1e-3, verbose=1)
sgdModelRoad = linear_model.SGDRegressor(loss='huber', penalty='l2', learning_rate='optimal', max_iter=1000, tol=1e-3, verbose=1)

In [742]:
sgdModelHome.fit(X_train_scaled, y_h)
sgdModelRoad.fit(X_train_scaled, y_r)

-- Epoch 1
Norm: 10.24, NNZs: 16, Bias: 107.223674, T: 1230, Avg. loss: 1.090794
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 10.92, NNZs: 16, Bias: 106.967056, T: 2460, Avg. loss: 0.365368
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 10.33, NNZs: 16, Bias: 107.180230, T: 3690, Avg. loss: 0.270865
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 10.40, NNZs: 16, Bias: 106.307606, T: 4920, Avg. loss: 0.225812
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 10.48, NNZs: 16, Bias: 107.383441, T: 6150, Avg. loss: 0.196967
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 10.73, NNZs: 16, Bias: 106.941008, T: 7380, Avg. loss: 0.182177
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 10.05, NNZs: 16, Bias: 106.746959, T: 8610, Avg. loss: 0.169150
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 10.16, NNZs: 16, Bias: 106.642009, T: 9840, Avg. loss: 0.155999
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 10.13, NNZs: 16, Bias: 107.235812, T: 11070, Avg. loss:

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='optimal',
       loss='huber', max_iter=1000, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=0.001, verbose=1,
       warm_start=False)

In [743]:
sgdModelHome.score(X_train_scaled, y_h)

0.932652728593592

In [744]:
sgdModelHome.coef_

array([ 1.17295361e+00,  1.64260881e-02,  1.17996462e-01,  4.40635868e-01,
       -4.53947460e-01,  1.02136905e+00,  2.81161861e+00, -1.86745246e-03,
        9.15880052e+00, -1.74148400e-02,  3.54961448e-02, -4.17117410e-01,
        5.21660109e-01, -4.93741248e-01,  2.81161861e+00, -7.30534031e-02])

In [745]:
sgdModelRoad.score(X_train_scaled, y_r)

0.9357847706960304

In [746]:
sgdModelRoad.coef_

array([ 9.88959082, -0.02127256,  0.1346479 , -0.44495899,  0.45436438,
       -0.49888495,  2.77767409,  0.0617991 ,  0.99408952,  0.02728823,
       -0.49575934,  0.41908346, -0.33108971,  1.00820198,  2.77767409,
       -0.07153348])

# Create Testing Data

In [574]:
# Url to get all boxscores for 2017-2018 regular season games
nba_18_19_trad_boxscores_url = "https://stats.nba.com/teams/boxscores-traditional/"
nba_18_19_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/"
nba_18_19_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/"

In [575]:
# visit website
browser.visit(nba_18_19_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df_adv = df[0].drop(columns=['Season'])
# Rename column names
df_adv.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df_adv['GameDate'] = df_adv['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_adv = df_adv.drop(['MIN'], axis=1)

In [576]:
# visit website
browser.visit(nba_18_19_fourfactor_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_ff = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%', 'OppeFG%', 'OppFTARate', 'OppTOV%', 'OppOREB%'], axis=1)

In [577]:
# visit website
browser.visit(nba_18_19_trad_boxscores_url)
time.sleep(10)
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
df_trad = df.drop(['MIN', 'MatchUp', 'W/L', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'PF', '+/-'], axis=1)

In [578]:
result_test = pd.merge(df_adv, df_ff, on=['Team', 'GameDate'])
result_test_final = pd.merge(result_test, df_trad, on=['Team', 'GameDate'])

In [579]:
result_test_sorted = result_test_final.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)

In [580]:
game_dates = result_test_sorted['GameDate'].unique()
game_dates = [game_day for game_day in game_dates]

In [581]:
temp_df = result_test_sorted.copy()

In [582]:
temp = db.nba_team_elo_testing.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)
elo_testing = temp[0]

In [583]:
date_index=0
newElo = {}
eloScores = []
homeTeam = []
roadTeam = []
for row in temp_df.itertuples():
    if(game_dates[date_index] != np.datetime64(row.GameDate)):
        #print("Update ELO")
        for key in newElo:
            elo_testing[key] = newElo[key]
        newElo = {}
        date_index += 1
    temp = row.MatchUp.split(' ')
    team1 = temp[0]
    team2 = temp[2]
    if temp[1] == 'vs.':
        if row._4 == 'W':
            outcome = True
        else:
            outcome = False
        eloUpdate, dummy = EloRating(elo_testing[team1], elo_testing[team2], 20, outcome)
        homeTeam.append(team1)
        roadTeam.append(team2)
    else:
        # Opposite since road team
        if row._4 == 'W':
            outcome = False
        else:
            outcome = True
        dummy, eloUpdate = EloRating(elo_testing[team2], elo_testing[team1], 20, outcome)
        roadTeam.append(team1)
        homeTeam.append(team2)
        
    newElo[row.Team] = eloUpdate
    eloScores.append(eloUpdate)
# Update for last date
for key in newElo:
    elo_testing[key] = newElo[key]
# Should update ELO Testing DB

In [584]:
db.nba_team_current_elo.drop()
db.nba_team_current_elo.insert_one(elo_testing)

In [585]:
temp_df['ELO'] = eloScores
temp_df['HomeTeam'] = homeTeam
temp_df['RoadTeam'] = roadTeam

In [586]:
home_df = temp_df[temp_df['Team'] == temp_df['HomeTeam']]
road_df = temp_df[temp_df['Team'] == temp_df['RoadTeam']]

In [587]:
game_df = pd.merge(home_df, road_df, how='inner', on=['HomeTeam', 'RoadTeam', 'GameDate'], suffixes=('_h', '_r'), validate="one_to_one")

In [588]:
game_df.columns

Index(['Team_h', 'MatchUp_h', 'GameDate', 'W/L_h', 'OffRtg_h', 'DefRtg_h',
       'NetRtg_h', 'AST%_h', 'AST/TO_h', 'ASTRatio_h', 'OREB%_h', 'DREB%_h',
       'REB%_h', 'TOV%_h', 'eFG%_h', 'TS%_h', 'PACE_h', 'PIE_h', 'FTARate_h',
       'PTS_h', 'ELO_h', 'HomeTeam', 'RoadTeam', 'Team_r', 'MatchUp_r',
       'W/L_r', 'OffRtg_r', 'DefRtg_r', 'NetRtg_r', 'AST%_r', 'AST/TO_r',
       'ASTRatio_r', 'OREB%_r', 'DREB%_r', 'REB%_r', 'TOV%_r', 'eFG%_r',
       'TS%_r', 'PACE_r', 'PIE_r', 'FTARate_r', 'PTS_r', 'ELO_r'],
      dtype='object')

In [589]:
game_df = game_df.drop(['Team_h', 'Team_r', 'MatchUp_h', 'MatchUp_r', 'W/L_h', 'W/L_r'], axis=1)

In [590]:
db.score_pred_testing_data.drop()

In [591]:
# export to json
items = game_df.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [592]:
db.score_pred_testing_data.insert_many(items_db)

In [747]:
X_test = game_df.loc[:,['DefRtg_h', 'PIE_h', 'FTARate_h', 'REB%_h', 'TOV%_h', 'TS%_h', 'PACE_h', 'ELO_h',
                        'DefRtg_r', 'PIE_r', 'FTARate_r', 'REB%_r', 'TOV%_r', 'TS%_r', 'PACE_r', 'ELO_r']]
y_h = game_df['PTS_h']
y_r = game_df['PTS_r']
y_test = game_df[['PTS_h', 'PTS_r']]

In [748]:
X_test.shape

(822, 16)

In [749]:
X_test_scaled = X_scaler.transform(X_test)

In [750]:
homeLinearModel.score(X_test_scaled, y_h)

0.93190068280913

In [752]:
roadLinearModel.score(X_test_scaled, y_r)

0.9285662208206512

In [756]:
huberModelHome.score(X_test_scaled, y_h)

0.9283792013835059

In [757]:
huberModelRoad.score(X_test_scaled, y_r)

0.9259077650087483

In [758]:
ardModelHome.score(X_test_scaled, y_h)

0.9317701564183263

In [760]:
ardModelRoad.score(X_test_scaled, y_r)

0.9275723253169956

In [761]:
sgdModelHome.score(X_test_scaled, y_h)

0.9275544090945826

In [762]:
sgdModelRoad.score(X_test_scaled, y_r)

0.9202836850863351

In [676]:
pred_hr = linearModel.predict(X_test_scaled)

In [677]:
pred_hr[0]

array([108.98532523, 100.67058843])

In [598]:
predictions_h = homeScorePredictor.predict(X_test_scaled)

In [599]:
predictions[0]

102.24064642117432

In [600]:
y_h[797]

92

In [601]:
predictions_r = roadScorePredictor.predict(X_test_scaled)

In [602]:
predictions_h[797]

92.22668598606127

In [603]:
predictions_r[797]

105.79102719566401

In [604]:
y_r[797]

105

In [344]:
#lakers_df = game_df[np.logical_or(game_df['HomeTeam'] == 'LAL', game_df['RoadTeam'] == 'LAL')]

In [605]:
lakers_df = game_df[game_df['RoadTeam'] == 'LAL']

In [606]:
indiana_df = game_df[game_df['HomeTeam'] == 'IND']

In [607]:
#result_test_sorted = result_test_final.sort_values(['GameDate'], ascending = 1).reset_index(drop=True)
lakers_df = lakers_df.sort_values(['GameDate'], ascending=0).reset_index(drop=True)
indiana_df = indiana_df.sort_values(['GameDate'], ascending=0).reset_index(drop=True)

In [611]:
laker_mean = lakers_df[2:7].mean()

In [612]:
laker_mean.iloc[2]

4.6000000000000005

In [613]:
indiana_mean = indiana_df[2:7].mean()

In [614]:
laker_mean['OffRtg_h']

111.88000000000002

In [615]:
my_test = pd.Series(data = [indiana_mean['OffRtg_h'], indiana_mean['PIE_h'], indiana_mean['FTARate_h'], indiana_mean['REB%_h'], indiana_mean['TOV%_h'], indiana_mean['TS%_h'], indiana_mean['PACE_h'], indiana_mean['ELO_h'],
                            laker_mean['OffRtg_r'], laker_mean['PIE_r'], laker_mean['FTARate_r'], laker_mean['REB%_r'], laker_mean['TOV%_r'], laker_mean['TS%_r'], laker_mean['PACE_r'], laker_mean['ELO_r']
                           ])
my_test

0      109.360000
1       48.700000
2        0.180800
3       48.380000
4       10.920000
5       53.240000
6       98.500000
7     1603.494912
8      107.300000
9       48.980000
10       0.263000
11      53.460000
12      15.240000
13      54.600000
14     103.394000
15    1483.664565
dtype: float64

In [417]:
my_test = pd.Series(data = [indiana_mean['PIE_h'], indiana_mean['FTARate_h'], indiana_mean['REB%_h'], indiana_mean['TOV%_h'], indiana_mean['TS%_h'], indiana_mean['PACE_h'], indiana_mean['ELO_h'],
                            laker_mean['PIE_r'], laker_mean['FTARate_r'], laker_mean['REB%_r'], laker_mean['TOV%_r'], laker_mean['TS%_r'], laker_mean['PACE_r'], laker_mean['ELO_r']
                           ])
my_test

0       48.700000
1        0.180800
2       48.380000
3       10.920000
4       53.240000
5       98.500000
6     1603.494912
7       48.980000
8        0.263000
9       53.460000
10      15.240000
11      54.600000
12     103.394000
13    1483.664565
dtype: float64

In [617]:
my_test = my_test.values.reshape(1,16)

In [618]:
X_my_scaled = X_scaler.transform(my_test)

In [619]:
homeScore = homeScorePredictor.predict(X_my_scaled)

In [620]:
homeScore[0]

103.20900750854561

In [621]:
roadScorePredictor.predict(X_my_scaled)[0]

110.33537179227459

In [622]:
lakers_df[0:1]['PIE_h']

0    53.4
Name: PIE_h, dtype: float64

In [623]:
homeBay.predict(X_my_scaled)[0]

111.09768358255634

In [624]:
roadBay.predict(X_my_scaled)

array([109.07844323])

In [784]:
def getHomeRoadMeans(teamRoad, teamHome):
    temp = db.score_pred_testing_data.find({'$or' : [{'HomeTeam': teamHome}, {'RoadTeam': teamRoad}]})
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    df_test = pd.DataFrame(temp)
    df_test = df_test.sort_values(['GameDate'], ascending=0).reset_index(drop=True)
    home_df = df_test[df_test['HomeTeam'] == teamHome].reset_index(drop=True)
    road_df = df_test[df_test['RoadTeam'] == teamRoad].reset_index(drop=True)
    homeMean = home_df[0:5].mean()
    roadMean = road_df[0:5].mean()
    test_series = pd.Series(data = [homeMean['DefRtg_h'], homeMean['PIE_h'], homeMean['FTARate_h'], homeMean['REB%_h'], homeMean['TOV%_h'], homeMean['TS%_h'], homeMean['PACE_h'], homeMean['ELO_h'],
                                    roadMean['DefRtg_r'], roadMean['PIE_r'], roadMean['FTARate_r'], roadMean['REB%_r'], roadMean['TOV%_r'], roadMean['TS%_r'], roadMean['PACE_r'], roadMean['ELO_r']
                           ])
    test_data = test_series.values.reshape(1,16)
    return X_scaler.transform(test_data)

In [785]:
X_test = getHomeRoadMeans('NYK', 'DET')

In [807]:
def predictAllModels(testData):
    predVals = {}
    #temp = [roadLinearModel.predict(testData), homeLinearModel.predict(testData)]
    #predVals['Linear'] = temp
    temp = [roadBay.predict(testData)[0], homeBay.predict(testData)[0]]
    mean = [temp[0], temp[1]]
    predVals['BayRidge'] = temp
    temp = [huberModelRoad.predict(testData)[0], huberModelHome.predict(testData)[0]]
    mean[0] += temp[0]
    mean[1] += temp[1]
    predVals['HuberRegress'] = temp
    temp = [ardModelRoad.predict(testData)[0], ardModelHome.predict(testData)[0]]
    mean[0] += temp[0]
    mean[1] += temp[1]
    predVals['ArdRegress'] = temp
    temp = [sgdModelRoad.predict(testData)[0], sgdModelHome.predict(testData)[0]]
    mean[0] += temp[0]
    mean[1] += temp[1]
    predVals['sgdRegress'] = temp

    predVals['modelMean'] = [mean[0]/4, mean[1]/4]
    return predVals

In [810]:
# Get test data and predict
X_test = getHomeRoadMeans('DEN', 'PHI')
testPred = predictAllModels(X_test)

In [811]:
testPred

{'BayRidge': [112.5343384306604, 117.2223506971034],
 'HuberRegress': [112.07015833425159, 117.58041332885983],
 'ArdRegress': [112.413239698532, 117.32698717221304],
 'sgdRegress': [112.52723027959867, 117.3208155080797],
 'modelMean': [112.38624168576065, 117.362641676564]}

In [ ]:
X_my_scaled = X_scaler.transform(X_test)

In [764]:
X_train.shape

(1230, 16)

In [644]:
homeBay.predict(X_my_scaled)[0]

110.54229701277073

In [645]:
roadBay.predict(X_my_scaled)[0]

97.63341059274505

In [503]:
import pickle
from sklearn.externals import joblib

In [504]:
scaler_filename = "bay_rig_scaler.save"
joblib.dump(X_scaler, scaler_filename)
bay_rig_home_filename = 'bay_rig_home_model.pkl'
pickle.dump(homeBay, open(bay_rig_home_filename, 'wb'))
bay_rig_road_filename = 'bay_rig_road_model.pkl'
pickle.dump(roadBay, open(bay_rig_road_filename, 'wb'))


In [537]:
teamHome = 'LAL'
teamRoad = 'LAC'
temp = db.score_pred_testing_data.find({'$or' : [{'HomeTeam': teamHome}, {'RoadTeam': teamRoad}]})
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df_test = pd.DataFrame(temp)
df_test = df_test.sort_values(['GameDate'], ascending=0).reset_index(drop=True)
home_df = df_test[df_test['HomeTeam'] == teamHome].reset_index(drop=True)
road_df = df_test[df_test['RoadTeam'] == teamRoad].reset_index(drop=True)

In [532]:
temp = home_df.head()['ELO_h']

In [538]:
home_df.iloc[0]['ELO_h']

1468.8400819865

In [529]:
temp

level_0       0.000000
index         3.000000
ELO_h      1468.840082
Name: 0, dtype: float64

In [540]:
bayScalar_filename = "bay_rig_scaler.save"
bay_rig_scaler = joblib.load(bayScalar_filename)

In [ ]:
bay_rig_scaler.scale_